In [ ]:
##log_dir = "logs/dicewars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# simulare partite contro altri bot 
# allenare il tuo agente
# salvare i pesi del modello (model.save())

import numpy as np
from tensorflow.summary import create_file_writer
import datetime
from importlib import import_module
from dicewars.match import Match
from dicewars.game import Game
from dicewars.player import RandomPlayer, AgressivePlayer, WeakerPlayerAttacker
from rl_agent import RLDicewarsAgent
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from collections import deque
from tqdm import tqdm
import wandb

run = wandb.init(
    project='Machine-Learning-DICEWARS',
    name='Dice Wars',
)
run_id = run.id 
api = wandb.Api()
run = api.run(f"Machine-Learning-DICEWARS/{run_id}")

if tf.config.list_physical_devices('GPU'):
    print("using cuda ok")
else:
    print('using cpu')

NUM_EPISODES = 1000
SAVE_MODEL_PATH = "./saved_models/dicewars_rl_model.keras"

# Crea directory se non esiste
os.makedirs(os.path.dirname(SAVE_MODEL_PATH), exist_ok=True)

def calculate_step_reward(prev_state, new_state, player_idx):
    """
    Reward intermedio basato su: conquiste, perdite, crescita di dadi
    """
    reward = 0

    # Aree controllate
    prev_areas = len(prev_state.player_areas[player_idx])
    new_areas = len(new_state.player_areas[player_idx])
    reward += (new_areas - prev_areas) * 0.3

    # Dadi totali
    prev_dice = prev_state.player_num_dice[player_idx]
    new_dice = new_state.player_num_dice[player_idx]
    reward += (new_dice - prev_dice) * 0.05

    # Penalità se ha fatto "end turn" senza attaccare
    if prev_areas == new_areas and prev_dice == new_dice:
        reward -= 0.2

    return reward

def calculate_final_reward(winner, player_idx):
    return 1.0 if winner == player_idx else -0.5


# Training loop
agent = RLDicewarsAgent()

## plots
win_history = []
reward_history = []
moving_avg = deque(maxlen=50)  # media mobile su ultimi 50 episodi


## training loop
for episode in tqdm(range(NUM_EPISODES), desc="Episode"):
    players = [agent] + [RandomPlayer() for _ in range(3)]
    game = Game(num_seats=4)
    match = Match(game)
    grid, state = match.game.grid, match.state
    epsilon = max(0.01, 0.1 - episode / NUM_EPISODES)  # decrescente

    history = []

    while match.winner == -1:
        current_player = state.player
        prev_state = state  # <- snapshot prima dell'azione

        if current_player == 0:
            action = agent.select_action(grid, state, epsilon=epsilon) # rimiuovi epsilon se non vuoi usare epsilon greedy
            state_vec = agent.encode_state(grid, state)
        else:
            action = players[current_player].get_attack_areas(grid, state)

        grid, state = match.step(action)

        if current_player == 0:
            reward = calculate_step_reward(prev_state, state, player_idx=0)
            history.append((state_vec, action, reward))


    # Final reward da partita
    final_reward = calculate_final_reward(match.winner, player_idx=0)
    episode_reward = sum([r for _, _, r in history]) + final_reward
    reward_history.append(episode_reward)

    # Registra vincita
    won = 1 if match.winner == 0 else 0
    win_history.append(won)
    moving_avg.append(won)

    # Allena
    for state_vec, action_taken, step_reward in history:
        total_reward = step_reward + final_reward
        agent.train_step(state_vec, action_taken, total_reward)

    print(f"Episode {episode + 1}/{NUM_EPISODES} | Reward: {episode_reward:.2f} | Win Rate: {np.mean(moving_avg):.2f}")
    wandb.log({'Episode': episode , 'Reward': episode_reward, 'Win Rate': np.mean(moving_avg)})
    # Salva modello ogni 50 episodi
    if (episode + 1) % 20 == 0:
        agent.save_model()
        

2025-03-27 14:11:31.112123: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 14:11:31.150912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-27 14:11:31.150933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-27 14:11:31.150959: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 14:11:31.158645: I tensorflow/core/platform/cpu_feature_g

using cuda ok
Inizializzazione agente RL


2025-03-27 14:11:41.576529: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13246 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5
2025-03-27 14:11:41.578944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13764 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:d8:00.0, compute capability: 7.5


[MODEL] Caricato da: ./saved_models/dicewars_rl_model.keras


Episode:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Episode 30/1000 | Reward: -1.55 | Win Rate: 0.23
Initializing player from standard library with name: RandomPlayer
Initializing player from standard library with name: RandomPlayer
Initializing player from standard library with name: RandomPlayer


Episode:   3%|▎         | 31/1000 [06:56<2:08:14,  7.94s/it]

Episode 31/1000 | Reward: -1.45 | Win Rate: 0.23
Initializing player from standard library with name: RandomPlayer
Initializing player from standard library with name: RandomPlayer
Initializing player from standard library with name: RandomPlayer


Episode:   3%|▎         | 32/1000 [07:02<1:59:48,  7.43s/it]

Episode 32/1000 | Reward: 1.80 | Win Rate: 0.22
Initializing player from standard library with name: RandomPlayer
Initializing player from standard library with name: RandomPlayer
Initializing player from standard library with name: RandomPlayer


In [15]:
#%load_ext tensorboard
#!kill 791914
#%tensorboard --logdir logs/dicewars